In [1]:
# check getloader return type supposed to be lr_image, hr_image
from model import SRResnetConfig, LitSRResnet
from dataset import get_dataloader

train_dataset,test_dataset =get_dataloader(batch_size=1, num_workers=1)


Files already downloaded and verified
Files already downloaded and verified


In [2]:
for x, y in train_dataset:
    print(x.shape, y.shape)
    break

torch.Size([1, 3, 128, 128]) torch.Size([1, 3, 224, 224])


In [3]:
# check model architecuture, whethere there are incopatible channel sizes
import torch
from model import *
config = SRResnetConfig()
model = LitSRResnet(config)

y = model(torch.rand(1, 3, 12, 12))
y.shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 31.15 GiB. GPU 0 has a total capacity of 14.75 GiB of which 13.84 GiB is free. Process 2591756 has 924.00 MiB memory in use. Of the allocated memory 787.68 MiB is allocated by PyTorch, and 20.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint


checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',   # Metric to monitor
    filename='model',  # Filename for the best checkpoint
    save_top_k=3,          # Save only the best model
    mode='min'             # Save the model with minimum validation loss
)

# train using lightning
trainer = L.Trainer(max_epochs=2, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_dataset, val_dataloaders=val_dataset)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


NameError: name 'model' is not defined